In [2]:
#Importing necessary libraries
!pip install transformers torch speechrecognition pydub



In [27]:
pip install  gtts playsound


  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7022 sha256=e87dc3d4b64fc70d6e8b561e447e9928290751421c37e67d3a44e8c8e46796c8
  Stored in directory: /Users/krish/Library/Caches/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound
Note: you may need to restart the kernel to use updated packages.


In [8]:

pip install pydub 


Note: you may need to restart the kernel to use updated packages.


In [1]:
import speech_recognition as sr
from pydub import AudioSegment

def transcribe_audio(file_path):
    # Convert the audio file to a format that SpeechRecognition can understand
    audio = AudioSegment.from_file(file_path)
    audio.export("converted.wav", format="wav")
    
    # Initialize the recognizer
    recognizer = sr.Recognizer()
    
    # Load the audio file
    with sr.AudioFile("converted.wav") as source:
        audio_data = recognizer.record(source)
    
    # Recognize the speech in the audio
    try:
        text = recognizer.recognize_google(audio_data)
        print(f'Transcript: {text}')
        return text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None


/Users/krish/anaconda3/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
from transformers import BertTokenizer, BertForSequenceClassification


/Users/krish/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
import torch
import speech_recognition as sr


In [4]:
from pydub import AudioSegment

In [5]:

# Define the transcribe_audio function
def transcribe_audio(file_path):
    audio = AudioSegment.from_file(file_path)
    audio.export("converted.wav", format="wav")
    
    recognizer = sr.Recognizer()
    with sr.AudioFile("converted.wav") as source:
        audio_data = recognizer.record(source)
    
    try:
        text = recognizer.recognize_google(audio_data)
        print(f'Transcript: {text}')
        return text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand the audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

In [6]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# Define the classify_intent function
def classify_intent(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()
    return predicted_class


In [8]:


# Define the intent labels
intent_labels = {0: 'Placing an Order', 1: 'Account Inquiries', 2: 'Technical Support'}

# Define the get_intent_label function
def get_intent_label(predicted_class):
    return intent_labels[predicted_class]

# Define the handle_input function
def handle_input(input_type, input_data):
    if input_type == 'audio':
        text = transcribe_audio(input_data)
        if text is None:
            return "Could not transcribe audio."
    elif input_type == 'text':
        text = input_data
    else:
        raise ValueError('Invalid input type')
    
    predicted_class = classify_intent(text)
    intent_label = get_intent_label(predicted_class)
    return intent_label

In [9]:


# Example of audio.wav
audio_file_path = 'audio.wav'
text_input = 'I want to place an order for a new laptop.'

# Handle audio input
intent_from_audio = handle_input('audio', audio_file_path)
print(f'Intent from audio: {intent_from_audio}')

# Handle text input
intent_from_text = handle_input('text', text_input)
print(f'Intent from text: {intent_from_text}')


Transcript: I want to know about my account
Intent from audio: Placing an Order
Intent from text: Placing an Order


In [10]:
pip install  gtts


Note: you may need to restart the kernel to use updated packages.


In [18]:
from gtts import gTTS
import os

from playsound import playsound

def text_to_speech(text, output_file="response.mp3"):
    tts = gTTS(text)
    tts.save(output_file)
    playsound(output_file)


In [13]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load pre-trained GPT-2 model and tokenizer for response generation
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

# Set padding token for GPT-2 tokenizer
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

def generate_response(intent, text):
    prompt = f"Intent: {intent}\nUser: {text}\nResponse:"
    inputs = gpt2_tokenizer.encode(prompt, return_tensors='pt', padding=True, truncation=True)
    
    # Create attention mask
    attention_mask = inputs.ne(gpt2_tokenizer.pad_token_id).long()
    
    outputs = gpt2_model.generate(
        inputs,
        attention_mask=attention_mask,
        max_length=100,
        num_return_sequences=1,
        pad_token_id=gpt2_tokenizer.eos_token_id
    )
    response = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [14]:
class ConversationContext:
    def __init__(self):
        self.history = []

    def update_context(self, user_input, intent, response):
        self.history.append({'user_input': user_input, 'intent': intent, 'response': response})

    def get_context(self):
        return self.history

context = ConversationContext()


In [15]:
# Define the handle_input function
def handle_input(input_type, input_data):
    if input_type == 'audio':
        text = transcribe_audio(input_data)
        if text is None:
            return "Could not transcribe audio."
    elif input_type == 'text':
        text = input_data
    else:
        raise ValueError('Invalid input type')
    
    predicted_class = classify_intent(text)
    intent_label = get_intent_label(predicted_class)
    
    response = generate_response(intent_label, text)
    context.update_context(text, intent_label, response)
    
    if input_type == 'audio':
        text_to_speech(response)
    else:
        print(f'Response: {response}')
    
    return intent_label, response


In [16]:

# Example usage
audio_file_path = 'audio.wav'
text_input = 'I want to place an order.'

In [19]:

# Handle audio input
intent_from_audio, response_from_audio = handle_input('audio', audio_file_path)
print(f'Intent from audio: {intent_from_audio}, Response: {response_from_audio}')


Transcript: I want to know about my account
Intent from audio: Placing an Order, Response: Intent: Placing an Order
User: I want to know about my account
Response: I want to know about your account
User: I want to know about your account
User: I want to know about your account
User: I want to know about your account
User: I want to know about your account
User: I want to know about your account
User: I want to know about your account
User: I want to know about your account
User:


In [20]:


# Handle text input
intent_from_text, response_from_text = handle_input('text', text_input)
print(f'Intent from text: {intent_from_text}, Response: {response_from_text}')

Response: Intent: Placing an Order
User: I want to place an order.
Response: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User:
Intent from text: Placing an Order, Response: Intent: Placing an Order
User: I want to place an order.
Response: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User: I want to place an order.
User:
